# Introduction

In this tutorial, you will learn how to read and plot netCDF4 data from the [Modern-Era Retrospective analysis for Research and Applications Version 2 (MERRA-2)](https://gmao.gsfc.nasa.gov/reanalysis/MERRA-2/). We will download a dataset and plot the world's two-meter air temperature.

# Import Dependencies
Let's import all the libraries we need. This needs to be done before any of the other cells can be run. These libraries were installed in the docker container you are using, so we will not need to worry about installing anything. Simply running the following cell takes care of all of the dependencies.

In [ ]:
from mpl_toolkits.basemap import Basemap, cm
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import netCDF4

# Approving the GES DISC DAAC
 
Before we can obtain any GES DISC data, we need to approve the GES DISC DAAC in our Earthdata Account. If you do not yet have an Earthdata Login Account, [you can create one here](https://urs.earthdata.nasa.gov). Next, we will need to approve the GES DISC data. Go to [this link](https://urs.earthdata.nasa.gov/approve_app?client_id=e2WVk8Pw6weeLUKZYOxvTQ) and click approve. If you are prompted to login, fill out your login information and click the link again.

# Downloading the Data

Let's use the *wget* command to download the dataset to our local machine. We will first change directories into the directory of this Jupyter Notebook. In your terminal, type the following command:

> `cd /home/condauser/tutorials/notebooks/MERRA_TUTORIAL/`

<br>
Now, you are in the same directory as this Jupyter Notebook.  Run the following command to list all files of this directory:

> `ls`

Next, we need to input your username and password for Earthdata login. Run the following command and enter your username and password when prompted:

> `read -p "enter your username: " username; read -s -p "enter your password: " password; echo ""`

<br> 
Now, let's run the following *wget* command to download the MERRA-2  *.nc4* file into this directory. We are obtaining this dataset from the GES DISC DAAC.  *wget* will require a URL for the data. We have provided that for you already. Later in this tutorial, we will learn where the data URL can be found. Run the following commands in your terminal.


> `cd /home/condauser/tutorials/notebooks/MERRA_TUTORIAL`

> `wget --user="$username" --password="$password" --no-check-certificate "https://goldsmr4.gesdisc.eosdis.nasa.gov/data/MERRA2_MONTHLY/M2TMNXSLV.5.12.4/2010/MERRA2_300.tavgM_2d_slv_Nx.201001.nc4"; username=""; password=""`

Now, the data will be downloaded as a netCDF4 file to your local machine. You can see this by typing in the following command:

> `ls`

You should now see `MERRA2_300.tavgM_2d_slv_Nx.201001.nc4` in your directory.

# Read the Data into Python

Let's use the *netCDF4* package to import the data into Python. To do so, we will use the `Dataset()` method which takes in as input the datafile.

In [ ]:
dataset = netCDF4.Dataset('MERRA2_300.tavgM_2d_slv_Nx.201001.nc4')

We can list all of the variables that are stored in this dataset by using the `variables` attribute.

In [ ]:
for var in dataset.variables:
    print(var)
    print("-"*40)


Let's grab `lon`, `lat`, and `T2M`.

In [ ]:
lon = dataset.variables["lon"]
lat = dataset.variables["lat"]
T2M = dataset.variables["T2M"]

In [ ]:
T2M

In this case, `T2M` refers to the mean 2-meter air temperature. We can get information about this variable by doing the following.

In [ ]:
T2M.long_name

In [ ]:
T2M.units

In [ ]:
T2M.shape

In [ ]:
T2M.dimensions

# Plotting the Data

We are going to use the packages *matplotlib*, *numpy*, and *basemap* to plot this data onto a world projection.

In [ ]:
%matplotlib inline

The above box ensures that any plots we create stay within the Jupyter Notebook.

## Reshape Data

To use the *basemap* package, the latitude, longitude and precipitation data must be the same numpy array shape. This is because basemap projects the data onto the the longitude and latitude coordinates.

In [ ]:
lon.ndim

In [ ]:
lon.shape

In [ ]:
lat.ndim

In [ ]:
lat.shape

In [ ]:
T2M.ndim

In [ ]:
T2M.shape

Even though `T2M` is three dimensions, the first dimensions corresponds to time, so we can just use one time stamp. We can grab the first time chunk using the following *numpy* syntax.

In [ ]:
T2M_t0 = T2M[0,:,:]

In [ ]:
T2M_t0.shape

Now, we need to reconfigure the coordinate data to a 2-dimensional grid. We can do so by using the *numpy* `meshgrid()` method. Specifically, the coordinate data must be a 2-dimensional grid with a shape of (361, 576) so that it matches the shape and dimensions of the temperature data. 

In [ ]:
lons, lats = np.meshgrid(lon, lat)

In [ ]:
lats.shape

Now, let's check the dimensions and shapes of our coordinates and temperature data.

In [ ]:
lats.shape == lons.shape == T2M_t0.shape

Below is an example of how meshgrid can be used.


In [ ]:
# example of np.meshgrid
xs = np.linspace(0,5,6)
ys = np.linspace(9,12,4)
print("xs")
print(xs, xs.shape)
print('\nys')
print(ys, ys.shape)
print('\nxgrid')
xgrid, ygrid = np.meshgrid(xs, ys)
print(xgrid, xgrid.shape)
print('\nygrid')
print(ygrid, ygrid.shape)

Now, we are ready to plot.

## Using Basemap

In [ ]:
plt.figure(figsize = (15,15))
plt.title("2-Meter Mean Air Temperature")
b = Basemap(projection="robin", lat_0=0, lon_0=0)
b.drawcoastlines()
cs = b.pcolor(lons, lats, T2M_t0, vmin=230, vmax=295, cmap=cm.jet, shading='flat',latlon=True)
cbar = b.colorbar(cs, location="bottom", pad="5%")
cbar.set_label("Temperature (K)")
plt.show()

## Saving the file

You can save the file to the same directory as the Jupyter Notebook using the following command. You can adjust the `dpi` to alter the quality of the image.

In [ ]:
plt.savefig('MERRA2_2m_airTemp_TEST.png', format='png', dpi=360)